In [2]:
import pandas as pd
feature1_path = '../final_feature/day_click_count.csv'
feature2_path = '../final_feature/day_statical.csv'

feature3_path = '../final_feature/evt_lbl_click_count.csv'
feature4_path = '../final_feature/hour_statical.csv'

feature5_path = '../final_feature/minute_statical.csv'
feature6_path = '../final_feature/weekday_click_count.csv'

feature1=pd.read_csv(feature1_path,sep='\t')
feature2=pd.read_csv(feature2_path,sep='\t')
feature3=pd.read_csv(feature3_path,sep='\t')
feature4=pd.read_csv(feature4_path,sep='\t')
feature5=pd.read_csv(feature5_path,sep='\t')
feature6=pd.read_csv(feature6_path,sep='\t')


In [3]:
train_agg_filename = r'../train/train_agg.csv'
train_flg_filename = r'../train/train_flg.csv'

test_agg_filename = r'../test/test_agg.csv'
test_flg_filename = r'../test/submit_sample.csv'

train_agg=pd.read_csv(train_agg_filename,sep='\t')
test_agg=pd.read_csv(test_agg_filename,sep='\t')

agg=pd.concat([train_agg,test_agg])

train_flg = pd.read_csv(train_flg_filename,sep='\t')
test_flg = pd.read_csv(test_flg_filename,sep='\t')
test_flg['FLAG'] = -1
del test_flg['RST']
flg = pd.concat([train_flg,test_flg],copy=False)

data = pd.merge(agg,flg,on=['USRID'],how='left',copy=False)

In [4]:
data=pd.merge(data,feature1,how='left')
data=pd.merge(data,feature2,how='left')
data=pd.merge(data,feature3,how='left')
data=pd.merge(data,feature4,how='left')
data=pd.merge(data,feature5,how='left')
data=pd.merge(data,feature6,how='left')

In [6]:
evt_one_hot = pd.read_csv('../final_feature/final_feature.csv',sep='\t')
data = pd.merge(data,evt_one_hot,how='left')

In [7]:
import pandas as pd
data1= pd.read_csv('../final_feature/lbl_one_shift.csv',sep='\t')
data=pd.merge(data,data1,how='left')

In [8]:
#加上day_dist_promote
feature7=pd.read_csv('../final_feature/day_dist_promote.csv')
feature7.head()

,USRID,dist_41_min,dist_41_max,dist_41_std,dist_41_mean,promote1,promote2
0,2,18,19,0.707107,18.500000,0,0
1,3,4,29,9.797959,14.666667,0,0
2,4,12,12,NaN,12.000000,0,0
3,5,12,27,7.505553,19.333333,0,0
4,7,4,23,7.092249,11.600000,0,0


In [9]:
feature7.fillna(0,inplace=True)

In [10]:
data.fillna(-1,inplace=True)

In [11]:
data = pd.merge(data,feature7,how='left')


In [14]:
train = data[data['FLAG']!=-1]
test = data[data['FLAG']==-1]
data.fillna(-1,inplace=True)
from feature_selector import FeatureSelector
fs = FeatureSelector(data = train.drop(['FLAG','USRID'],axis=1), labels = train['FLAG'])

In [15]:
fs.identify_collinear(correlation_threshold = 0.98)

330 features with a correlation magnitude greater than 0.98.



In [16]:
train.shape

(80000, 625)

In [17]:
fs.identify_zero_importance(task = 'classification', 
 eval_metric = 'auc', 
 n_iterations = 10, 
 early_stopping = True)
# list of zero importance features
zero_importance_features = fs.ops['zero_importance']


Training Gradient Boosting Model

Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[134]	valid_0's auc: 0.858004
Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[236]	valid_0's auc: 0.853306
Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[136]	valid_0's auc: 0.835557
Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[121]	valid_0's auc: 0.86081
Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[140]	valid_0's auc: 0.867262
Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[153]	valid_0's auc: 0.861855
Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[162]	valid_0's auc: 0.855839
Train until valid scores didn't improve in 100 rounds.
Early stopping, best iteration is:
[127]	valid_0's auc: 0.84834
Train un

In [18]:
train_removed_all = fs.remove(methods = 'all')
columns=list(train_removed_all.columns)
columns.append('USRID')
columns.append('FLAG')


['collinear', 'zero_importance'] methods have been run

Removed 359 features.


In [19]:
data =data[columns]

In [ ]:

train=data[data['FLAG']!=-1]
test = data[data['FLAG']==-1]

import numpy as np
import pandas as pd
from pandas import DataFrame
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
import sys
import scipy as sp
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
train.fillna(-1,inplace=True)
for rnd in [100]:   
    train_x = train.drop(['USRID', 'FLAG'], axis=1).values
    train_y = train['FLAG'].values
    auc_list = []
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
    for train_index, test_index in skf.split(train_x, train_y):
        print('Train: %s | test: %s' % (train_index, test_index))
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
    
        params = {'booster': 'gbtree',
              'objective':'binary:logistic',
              'eta': 0.02,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 9,  # 2 3
              'silent':1,
              'seed':rnd,
            'eval_metric':'auc'
              }
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_test,y_test)
        dvali = xgb.DMatrix(X_test)
        model = xgb.train(params, dtrain, evals=[(dtrain,'dtrain'),(dval,'dval'),],early_stopping_rounds=100,verbose_eval=300,num_boost_round=3000)
        pred_value = model.predict(dvali)
    
        pred_value = np.array(pred_value)
        pred_value = [ele + 1 for ele in pred_value]
    
        y_test = np.array(y_test)
        y_test = [ele + 1 for ele in y_test]
    
        fpr, tpr, thresholds = roc_curve(y_test, pred_value, pos_label=2)
            
        auc = metrics.auc(fpr, tpr)
        print('auc value:',auc)
        auc_list.append(auc)
    
    print('validate result:',np.mean(auc_list))
    






In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
import sys
import scipy as sp
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

In [ ]:
rnd=100
data.fillna(-1,inplace=True)
train =data[data['FLAG']!=-1]
test =data[data['FLAG']==-1]
columns=list(train_removed_all.columns)
columns.append('USRID')
columns.append('FLAG')

train=train[columns]
test = test[columns]

train_x = train.drop(['USRID', 'FLAG'], axis=1).values
train_y = train['FLAG'].values
dtrain = xgb.DMatrix(train_x, train_y)
test_x = test.drop(['USRID','FLAG'],axis=1).values

params = {'booster': 'gbtree',
              'objective':'binary:logistic',
              'eta': 0.02,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 9,  # 2 3
              'silent':1,
              'seed':rnd,
            'eval_metric':'auc'
              }
dvali = xgb.DMatrix(test_x)
model = xgb.train(params, dtrain, evals=[(dtrain,'dtrain'),],verbose_eval=100,num_boost_round=730)




pred_value = model.predict(dvali)
result = test[['USRID']]

result['RST'] =pred_value
result.to_csv('../submit/7-10-xgb_select_all-test.csv',index=None,sep='\t')